In [1]:
import torch
from data import ClimateDataset
from models import ClassificationModel
from engine import Engine

In [2]:
model_checkpoint='roberta-large'

In [3]:
results = {'acc':[],
           'f1':[],
           'g':[],
           'wd':[]}

In [4]:
lr = 1.0e-5
gamma = 2
wd = 0.01

print(f'Grid search {model_checkpoint}, lr: {lr} ,focal loss, gamma: {gamma}')
data = ClimateDataset(model_to_train=1,model_checkpoint=model_checkpoint,batch_size=16)
data.setup_dataloaders()
model = ClassificationModel(model_checkpoint=data.model_checkpoint,num_labels=data.num_labels)
trainer = Engine(epochs=10,labels=data.labels)
trainer.model = model.model
trainer.dataset_encoded = data.dataset_encoded
acc, f1 = trainer.run(lr=lr,
                        wd=wd,
                        train_dataloader=data.train_dataloader,
                        eval_dataloader=data.eval_dataloader,
                        test_dataloader=data.test_dataloader,
                        focalloss=True,
                        early_stop=3,
                        accumulation_steps=2,
                        gamma=gamma)
results['acc'].append(acc)
results['f1'].append(f1)
results['g'].append(gamma)
results['wd'].append(wd)
print('### '*10)
print(results)
print('### '*10)

Grid search roberta-large, lr: 1e-05 ,focal loss, gamma: 2


Map:   0%|          | 0/1796 [00:00<?, ? examples/s]

Map:   0%|          | 0/256 [00:00<?, ? examples/s]

Map:   0%|          | 0/457 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


1 / 10: Train Loss:	0.1740	Val Loss:	0.5718	Accuracy:	0.8228	F1:	0.7640
2 / 10: Train Loss:	0.1222	Val Loss:	0.5363	Accuracy:	0.7790	F1:	0.7515
3 / 10: Train Loss:	0.0814	Val Loss:	0.3781	Accuracy:	0.8271	F1:	0.7970
4 / 10: Train Loss:	0.0575	Val Loss:	0.3598	Accuracy:	0.8337	F1:	0.7994
5 / 10: Train Loss:	0.0323	Val Loss:	0.4431	Accuracy:	0.7987	F1:	0.7747
6 / 10: Train Loss:	0.0256	Val Loss:	0.4895	Accuracy:	0.7899	F1:	0.7664
7 / 10: Train Loss:	0.0179	Val Loss:	0.3341	Accuracy:	0.8687	F1:	0.8181
8 / 10: Train Loss:	0.0108	Val Loss:	0.3284	Accuracy:	0.8709	F1:	0.8206
9 / 10: Train Loss:	0.0135	Val Loss:	0.3381	Accuracy:	0.8621	F1:	0.8005
10 / 10: Train Loss:	0.0052	Val Loss:	0.3426	Accuracy:	0.8753	F1:	0.8310
best (higgest macro f1-score) val results:
              precision    recall  f1-score   support

       bknow       0.80      0.70      0.74       119
      struct       0.90      0.94      0.92       338

    accuracy                           0.88       457
   macro avg      